In [2]:
from agents import Agent, function_tool, Runner

In [3]:
import requests

In [13]:
# def fetch_webpage_tool(url):
#     jina_reader_base_url = "https://r.jina.ai/"
#     jina_reader_url = jina_reader_base_url + url
#     response = requests.get(jina_reader_url)
#     return response.content.decode('utf-8')

### search agent

In [18]:
from minsearch import AppendableIndex

In [19]:
index = AppendableIndex(text_fields=['content'])

In [31]:
from typing import Any, Optional, Dict, List
from requests.exceptions import RequestException

class SearchTools:
    def __init__(self, index: Any) -> None:
        """
        Initialize the SearchTools with an index object.

        Args:
            index (Any): An index object or a list to store chunks.
                         Must implement `append(chunk)` if it's a list,
                         or `search(query, num_results)` if a search index.
        """
        self.index = index

    def index_page(self, url: str, content: Optional[str] = None) -> bool:
        """
        Fetch and index the content of a web page.

        Args:
            url (str): The URL of the web page.
            content (Optional[str]): If provided, use this content instead of fetching.

        Returns:
            bool: True if indexing succeeds, False otherwise.
        """
        try:
            if not content:
                content = fetch_webpage_tool(url)
                if not content:
                    print(f"No content found at the URL: {url}")
                    return False

            # Chunk the content using a sliding window approach
            chunks: List[Dict[str, Any]] = docs.sliding_window(content, window_size=3000, step=1000)
            for chunk in chunks:
                chunk["url"] = url

                # Append to index; works if index is a list or supports append
                if hasattr(self.index, "append"):
                    self.index.append(chunk)
                else:
                    raise AttributeError("Index object does not support 'append' method.")

            return True

        except (RequestException, ValueError) as e:
            print(f"Error indexing web page {url}: {e}")
            return False
        except Exception as e:
            print(f"Unexpected error during indexing: {e}")
            return False

    def search(self, query: str) -> Dict[str, Any]:
        """
        Search for relevant results in the indexed content.

        Args:
            query (str): The user's search query.

        Returns:
            Dict[str, Any]: A dictionary containing the top search results, or empty if search fails.
        """
        try:
            if hasattr(self.index, "search"):
                results = self.index.search(query, num_results=5)
                return results
            else:
                raise AttributeError(
                    "Index object must implement a 'search(query, num_results=...)' method."
                )
        except Exception as e:
            print(f"Error performing search: {e}")
            return {}


#### summary agent

In [24]:
import requests
from requests.exceptions import RequestException

def fetch_webpage_tool(url: str) -> str:
    """
    Fetch the HTML content of a webpage using the Jina reader base URL.

    Args:
        url (str): The path or endpoint to append to the Jina reader base URL.

    Returns:
        str: The decoded HTML content of the fetched webpage.

    Raises:
        ValueError: If the URL is empty or None.
        RuntimeError: If there is a network issue or the request fails.
    """
    if not url:
        raise ValueError("The URL parameter must not be empty or None.")

    jina_reader_base_url = "https://r.jina.ai/"
    jina_reader_url = jina_reader_base_url + url

    try:
        response = requests.get(jina_reader_url)
        response.raise_for_status()  # Raise HTTPError for bad responses
    except RequestException as e:
        raise RuntimeError(f"Failed to fetch the webpage: {e}") from e

    try:
        content = response.content.decode('utf-8')
    except UnicodeDecodeError as e:
        raise RuntimeError(f"Failed to decode webpage content: {e}") from e

    return content


In [22]:
content = fetch_webpage_tool("https://en.wikipedia.org/wiki/Capybara")
print(content)

Title: Capybara

URL Source: https://en.wikipedia.org/wiki/Capybara

Published Time: 2001-10-13T20:04:37Z

Markdown Content:
[![Image 1: This is a good article. Click here for more information.](https://upload.wikimedia.org/wikipedia/en/thumb/9/94/Symbol_support_vote.svg/20px-Symbol_support_vote.svg.png)](https://en.wikipedia.org/wiki/Wikipedia:Good_articles* "This is a good article. Click here for more information.")

[![Image 2: Page semi-protected](https://upload.wikimedia.org/wikipedia/en/thumb/1/1b/Semi-protection-shackle.svg/20px-Semi-protection-shackle.svg.png)](https://en.wikipedia.org/wiki/Wikipedia:Protection_policy#semi "This article is semi-protected until March 17, 2028 at 19:23 UTC.")

From Wikipedia, the free encyclopedia

| Capybara |
| --- |
| [![Image 3](https://upload.wikimedia.org/wikipedia/commons/thumb/3/34/Hydrochoeris_hydrochaeris_in_Brazil_in_Petr%C3%B3polis%2C_Rio_de_Janeiro%2C_Brazil_09.jpg/250px-Hydrochoeris_hydrochaeris_in_Brazil_in_Petr%C3%B3polis%2C_Rio_d

In [17]:
summary_instructions = """
You are an intelligent assistant with access to two tools: 

1. **Fetch Web Page** - Fetches the full content of a webpage.  
   Usage: fetch_webpage_tool(url: str) -> str  

2. **Save Summary** - Saves a summary of a text.  
   Usage: save_summary_tool(summary: str) -> None  

Your task is to answer user questions about webpages. Follow these steps:  

1. When asked "What is this page about?" first use the **Fetch Web Page** tool with the provided URL to get the page content.  
2. Read the fetched content and generate a concise summary of what the page is about.  
3. Use the **Save Summary** tool to save the summary.  
4. Return the summary as your answer to the user.  

Always ensure the summary is clear, concise, and accurate. Do not skip any of the steps.
"""

In [ ]:
summary_agent = Agent(
    name = 'summary_agent',
    instructions = summary_instructions,
    handoff_description = 